In [1]:
import pandas as pd

In [2]:
file_path = 'cleaned_data.csv'
df = pd.read_csv(file_path)
df.head()

,Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6,Comment.7,Comment.8,Comment.9,...,Comment.52,Comment.53,Comment.54,Comment.55,Comment.56,Comment.57,Comment.58,Environment,Summary,Description
0,I am running the command provided by the docum...,Hi [~accountid:6213cf7cc345490071968be0] The ...,"Ok, thanks for your reply. \n\nI had seen this...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,version we are running is: Artifactory Online ...,plugin config-import not found for jfrog Artif...,I am following the steps from [https://docs.jf...
1,[https://jfrog.com/help/r/jfrog-release-inform...,*According to RTDEV-5069:*\n\nThe feature is s...,[~accountid:5f79a75eac3a2d006ffcc8b7] : \n\n* ...,[~accountid:5f79a75eac3a2d006ffcc8b7] We have ...,Since Composer *Virtual* Repository supports o...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RTDEV-5069 / mirroring https://asset-packagist...,We are running Artifactory 7.63.14 and have pr...
2,This is the current value I would like to chan...,[https://jfrog.atlassian.net/browse/RTFACT-154...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Developement and Production,Docker - Repository Default URL,Similar to Issue: [https://jfrog.atlassian.net...
3,Ik can get the manifest from the remote repo w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Artifactory-pro 7.63.14 within a docker conta...,getting manifest unknown error pulling image,I am getting the following error when pulling ...
4,Corresponding Log record from nginx reverse pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Linux centos,Caught exception while saving incoming stream ...,We have recently upgrade Artifactory to 7.63.1...


In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [4]:
embeddings = []

# Iterate through each row of the column
for i in df['Summary']:
    # Encode the sentence using the SBERT model
    embedding = model.encode(i)
    # Append the embedding to the list
    embeddings.append(embedding)

df['embeddingsSummary'] = embeddings

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df['Summary'][700]

'Authentication  problem'

In [7]:
df['Description'][700]

'Hi,\nwe had some problems with our artifactory when the user access a virtual repo which has an local repo who you need an authenticated user.\n\n\n* virtual repo\n**  local repo 1\n**  local repo 2\n**  external repo 1\n** external repo 2\n\t\n\nPermissions Management:\n - Any Remote (on any remote repo): anonymous -> deploy\n - local-repo1-permission (local repo 1): reader-user -> read, deployer-user -> delete, deploy, annotate, read\n - local-repo2-permission (local repo 2): reader-user -> read, deployer-user -> read\n\nThe problem occur when the needed jar is in "local repo 1" and your credential are from one of the both users'

In [8]:
df['Comment'][700]

'Security General Settings :\n- Allow Anonymous Access\n'

In [9]:
df['Comment.2'][1]

'[~accountid:5f79a75eac3a2d006ffcc8b7] : \n\n* _Store Artifacts Locally_ is enabled\n* we use the repository directly\n* but: as far as i know [https://asset-packagist.org|https://asset-packagist.org] does not support v2, only v1. We will check…'

In [10]:
def clean_comment(comment):
  try:
    end_index = comment.find(']') + 1
    return comment[end_index:].strip()  # Remove leading/trailing whitespace
  except ValueError:
    return comment

In [11]:
for col in df.filter(like='Comment'):
  df[col] = df[col].astype(str)
  df[col] = df[col].apply(clean_comment)

In [12]:
def merge_comments(df):

  comment_cols = ['Comment', 'Comment.1', 'Comment.2', 'Comment.3', 'Comment.4', 'Comment.5', 'Comment.6', 'Comment.7', 'Comment.8', 'Comment.9', 'Comment.10', 'Comment.11', 'Comment.12', 'Comment.13', 'Comment.14', 'Comment.15', 'Comment.16', 'Comment.17', 'Comment.18', 'Comment.19', 'Comment.20', 'Comment.21', 'Comment.22', 'Comment.23', 'Comment.24', 'Comment.25', 'Comment.26', 'Comment.27', 'Comment.28', 'Comment.29', 'Comment.30', 'Comment.31', 'Comment.32', 'Comment.33', 'Comment.34', 'Comment.35', 'Comment.36', 'Comment.37', 'Comment.38', 'Comment.39', 'Comment.40', 'Comment.41', 'Comment.42', 'Comment.43', 'Comment.44', 'Comment.45', 'Comment.46', 'Comment.47', 'Comment.48', 'Comment.49', 'Comment.50', 'Comment.51', 'Comment.52', 'Comment.53', 'Comment.54', 'Comment.55', 'Comment.56', 'Comment.57', 'Comment.58']

  # filter non-NaN comments
  def filter_comments(row):
    return [comment for comment in row[comment_cols] if pd.notna(comment)]

  # Create a new column 'combined_comments' to store merged comments
  df['Comments'] = df.apply(filter_comments, axis=1)

  # Iterate through rows and build comment lists
  for index, row in df.iterrows():
    comments = [comment for comment in row[comment_cols] if pd.notnull(comment)]
    df.at[index, 'Comments'] = comments

  return df

df = merge_comments(df.copy())

df.head()


,Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6,Comment.7,Comment.8,Comment.9,...,Comment.54,Comment.55,Comment.56,Comment.57,Comment.58,Environment,Summary,Description,embeddingsSummary,Comments
0,I am running the command provided by the docum...,The Config Import plugin in the JFrog SaaS ins...,or the command “jf plugin install config-impor...,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,version we are running is: Artifactory Online ...,plugin config-import not found for jfrog Artif...,I am following the steps from [https://docs.jf...,"[0.0066620787, 0.12316594, -0.0076982556, 0.08...",[I am running the command provided by the docu...
1,"{quote}RTDEV-5069 Fixed an issue whereby, Arti...",it that relevant to the situation you described?,: \n\n* _Store Artifacts Locally_ is enabled\n...,"We have checked this, from our point of view [...",Since Composer *Virtual* Repository supports o...,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,NaN,RTDEV-5069 / mirroring https://asset-packagist...,We are running Artifactory 7.63.14 and have pr...,"[-0.08578057, 0.063168734, -0.0045067403, 0.17...","[{quote}RTDEV-5069 Fixed an issue whereby, Art..."
2,This is the current value I would like to chan...,,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,Developement and Production,Docker - Repository Default URL,Similar to Issue: [https://jfrog.atlassian.net...,"[-0.0026910885, 0.054876674, -0.013277466, -0....",[This is the current value I would like to cha...
3,"{\n ""schemaVersion"": 2,\n ""mediaType"": ""ap...",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,Artifactory-pro 7.63.14 within a docker conta...,getting manifest unknown error pulling image,I am getting the following error when pulling ...,"[-0.10222563, -0.027658874, -0.005382626, 0.11...","[{\n ""schemaVersion"": 2,\n ""mediaType"": ""a..."
4,"""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""py...",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,Linux centos,Caught exception while saving incoming stream ...,We have recently upgrade Artifactory to 7.63.1...,"[-0.0813854, 0.14918321, -0.007328004, 0.02076...","[""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""p..."


In [13]:
sum_input_sen = df['Summary'][700]
des_input_sen = df['Description'][700]

In [17]:
df["embeddingsSummary"] = df["embeddingsSummary"].tolist()

In [14]:
# Function to calculate cosine similarity between sentences
def calculate_similarity(input_sentence, embeddings):
  input_embedding = model.encode(input_sentence)

  similarities = []
  for embedding in embeddings:
    similarity = cosine_similarity(input_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
    similarities.append(similarity)

  return similarities

In [15]:
# Calculate similarity scores
sum_similarities = calculate_similarity(sum_input_sen, df["embeddingsSummary"])

# Display results
for i, row in df.iterrows():
  Summary = row["Summary"]
  similarity = sum_similarities[i]
  print(f"Summary: {Summary}, Similarity: {similarity:.4f}")

Summary: plugin config-import not found for jfrog Artifactory cloud implementation, Similarity: 0.1646
Summary: RTDEV-5069 / mirroring https://asset-packagist.org, Similarity: 0.2443
Summary: Docker - Repository Default URL, Similarity: 0.1919
Summary: getting manifest unknown error pulling image, Similarity: 0.2011
Summary: Caught exception while saving incoming stream to file Artifactory version 7.63.14, Similarity: 0.1888
Summary: Dot in remote repository url not working anymore, Similarity: 0.2110
Summary: Generic Remotes do not respect TTL, Similarity: 0.2119
Summary: Updating properties through the Web UI causes system to crash when the update contains a large set of artifacts, Similarity: 0.2261
Summary: The same include pattern works for authenticated users but not for an anonymous user for PyPi repositories, Similarity: 0.5607
Summary: Search order mvn index no longer customizable, Similarity: 0.0014
Summary: Build info is missing from the results of AQL when artifacts cached 

In [20]:
len(sum_similarities)

9419

In [16]:
# Store summaries with similarity > 0.7 and their scores in a list
similar_summaries_and_scores = []
for summary, similarity in zip(df["Summary"], sum_similarities):
  if similarity > 0.5:
    similar_summaries_and_scores.append((summary, similarity))

# Sort similar summaries by similarity score (highest first)
sorted_similar_summaries_and_scores = sorted(similar_summaries_and_scores, key=lambda x: x[1], reverse=True)

# Print top 5 similar sentences and their scores
for i, (summary, similarity) in enumerate(sorted_similar_summaries_and_scores):
  if len(sum_similarities) == 0:
    print('No similar tickets found.')
  if i == 5:
    break  # Print only the top 5
  print(f"Similar summary: {summary} \n , Similarity: {similarity:.4f} \n")

Similar summary: Authentication  problem 
 , Similarity: 1.0000 

Similar summary: Authentication 
 , Similarity: 0.9101 

Similar summary: Purpose of Enable Token Authentication unclear 
 , Similarity: 0.7027 

Similar summary: I would to use a certificate as an authentication option 
 , Similarity: 0.6616 

Similar summary: Support Certificate based authentication for replication between Artifactory instances 
 , Similarity: 0.6579 



In [17]:
len(similar_summaries_and_scores)

49

In [38]:
data = []

for i, (summary, similarity) in enumerate(zip(df['Summary'], sum_similarities)):
    if similarity > 0.5:
        # new_df = new_df.append({
            data.append({
            'Summary': summary,
            'EmbeddingsSummary': embeddings[i],
            'Description': df.loc[i, 'Description'],
            'Comments': df.loc[i, 'Comments']
            })

new_df = pd.DataFrame.from_records(data)

In [33]:
db = pd.DataFrame()
db['Summary'] = df['Summary']
db['EmbeddingsSummary'] = df['embeddingsSummary']
db['Description'] = df['Description']
db['EmbeddingsDescription'] = df['embeddingsDescription']
db['Comments'] = df['Comments']

# db.to_csv('df_for_WebApp.csv', sep=',', index=False, encoding='utf-8')

In [21]:
new_df.shape

(49, 4)

In [35]:
new_df.head()

,Summary,EmbeddingsSummary,Description,Comments
0,The same include pattern works for authenticat...,"[-0.0046160882, 0.20808952, -0.007845477, 0.02...",*Problem description:* \n\nWhen working with t...,[This is misconfiguration (the user was part o...
1,Add additional authenticaion methods to the Py...,"[-0.14333433, 0.29187408, -0.009490655, -0.009...",The PyPi Set Me Up dialogue suggestion is to p...,"[,\n.netrc is a generic file and not specific ..."
2,Invalid authentication removes groups from Cro...,"[0.06753627, 0.10480404, -0.005775153, 0.07874...",h3. Steps to reproduce\r\n # Configure Artifac...,[Closed this ticket as this is the desired beh...
3,"Mail Server not working, with error 535 Authen...","[0.003318736, 0.08931374, -0.004173661, -0.006...",*Steps to reproduce*\r\n * In the Mail Server ...,"[I just tested with version *7.38.10*, and the..."
4,Authenticating to Artifactory Crate Package Re...,"[-0.10437466, 0.17522216, -0.0049882694, 0.125...",h2. Background\r\n\r\nPreviously we had a bug ...,[The Cargo client does not send any authentica...


In [25]:
df['Description'] = df['Description'].astype(str)

In [39]:
embeddings = []

for i in new_df['Description']:
    # Encode the sentence using the SBERT model
    embedding = model.encode(i)
    embeddings.append(embedding)

new_df['embeddingsDescription'] = embeddings

In [40]:
new_df["embeddingsDescription"] = new_df["embeddingsDescription"].tolist()

In [41]:
# Calculate similarity scores
des_similarities = calculate_similarity(des_input_sen, new_df["embeddingsDescription"])

for i, row in new_df.iterrows():
  Description = row["Description"]
  similarity = des_similarities[i]
  print(f"Description: {Description}, Similarity: {similarity:.4f}")

Description: *Problem description:* 

When working with the PyPi repository, the same include pattern works for authenticated users but not for an anonymous user.

*What is the impact:*

Even if the provided pattern matches, anonymous users cannot install packages.

*What is the expected behavior?*

The anonymous user can install packages using the same included pattern as authenticated users.

*Steps to reproduce:*

# Create a new local PyPi repository.
# Upload any Pypi package to the local repository.
# Create a new permission target with the include pattern set to '<packagename>**'. Additionally, add an anonymous user with write permissions to this repository and another user for testing. Ensure that the anonymous user does not inherit permissions from any other permission targets.
# Set up the pip client using the 'Set Me Up' option, ensuring that the pip.conf file does not contain the username and password in the URL, allowing for anonymous usage of the pip client.
# Make sure an

In [42]:
len(des_similarities)

49

In [43]:
# Store descriptions with similarity > 0.7 and their scores in a list
similar_descriptions_and_scores = []
for Description, similarity in zip(new_df["Description"], des_similarities):
  if similarity > 0.5:
    similar_descriptions_and_scores.append((Description, similarity))

# Sort similar descriptions by similarity score (highest first)
sorted_similar_descriptions_and_scores = sorted(similar_descriptions_and_scores, key=lambda x: x[1], reverse=True)

# Print top 5 similar sentences and their scores
for i, (Description, similarity) in enumerate(sorted_similar_descriptions_and_scores):
  if i == 5:
    break  # Print only the top 5
  print(f"Similar Description: {Description} \n , Similarity: {similarity:.4f} \n")

Similar Description: Hi,
we had some problems with our artifactory when the user access a virtual repo which has an local repo who you need an authenticated user.


* virtual repo
**  local repo 1
**  local repo 2
**  external repo 1
** external repo 2
	

Permissions Management:
 - Any Remote (on any remote repo): anonymous -> deploy
 - local-repo1-permission (local repo 1): reader-user -> read, deployer-user -> delete, deploy, annotate, read
 - local-repo2-permission (local repo 2): reader-user -> read, deployer-user -> read

The problem occur when the needed jar is in "local repo 1" and your credential are from one of the both users 
 , Similarity: 1.0000 

Similar Description: Good day,

We would like your assistance to check on a possible bug with JFrog Artifactory 7.4.

We are having a hard time configuring zypper clients (SLES and OpenSUSE) to work with our RPM Repository due to some "authentication" issues. At first, we thought it was just an issue with zypper and opensuse so we

In [44]:
len(similar_descriptions_and_scores)

4

In [45]:
similar_descriptions_and_scores[1]

('Problem:\r\n\r\nOne of my dev teams uses a paid-for package (specifically, the FontAwesome Pro package) that I would like to mirror as a remote repository. However, the FontAwesome Pro package can only be retrieved from\xa0a private NPM registry which requires an authToken (that they have provided us with) to verify access and allow downloading of the package. I am unable to add their private registry to our Artifactory instance as a remote, due to the fact that Artifactory does not appear to support authentication to a NPM\xa0registry via authToken -- only username:password or SSL/TLS authentication options appear to be supported by Artifactory.\r\n\r\n\xa0\r\n\r\nMore info:\r\n\r\n[https://docs.npmjs.com/about-authentication-tokens]\r\n\r\nAs of 5.5.1, NPM has supported using an authToken to set and manage access to private repos or packages.\r\n\r\nFontAwesome previously used a single URL for both the registry address and the authentication but they\'ve moved away from that in fav

### Comments

In [46]:
ss_model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')

In [35]:
query_embedding = ss_model.encode(des_input_sen)

In [36]:
comments_for_similar_descriptions = []

for description, similarity in similar_descriptions_and_scores:

  # Find the index of the matching description in the original DataFrame
  description_index = new_df[new_df['Description'] == description].index[0]

  # Extract the corresponding comment
  comment = new_df.loc[description_index, 'Comments']

  # Append a tuple (description, similarity, comment) to the comments list
  comments_for_similar_descriptions.append((comment))

In [37]:
emb_coms = []
for comment in comments_for_similar_descriptions :
  emb_com = ss_model.encode(comment)
  emb_coms.append(emb_com)


In [38]:
query_embedding = ss_model.encode(des_input_sen)

s_search = []
for i in emb_coms:
  ss_score = util.semantic_search(query_embedding.reshape(1, -1), i)
  s_search.append(ss_score)

In [39]:
s_search1 = []
for i in s_search:
  for j in i :
    s_search1.append(i)

In [41]:
len(comments_for_similar_descriptions)

4

In [40]:
for i, sublist in enumerate(s_search):
  print(f"Sentence {i+1} similar descriptions:")
  for item in sublist:
    for j in item :
      corpus_id = j['corpus_id']
      score = j['score']
      comment = comments_for_similar_descriptions[i][corpus_id]  # Access comment by corpus_id
      print(f"\t Score: {score:.2f}, \n Comment: {comment}")

Sentence 1 similar descriptions:
	 Score: 0.65, 
 Comment: We're seeing similar behaviour on version  7.25.7.  Sometimes Artifactory responds with 403 on a certain package that is otherwise fetched just fine.  The package is fetch in a CI job, and the credentials are unchanged.  After a little while it will start working again.
	 Score: 0.58, 
 Comment: We ran into an issue that looks similar to this and discovered that the problem appears to be some kind of cache inside Artifactory. The following can be used to reliably trigger the issue for us:
 # Download a file via `curl --user "username:password" https://artifactory.example.com/foo/bar` to verify that the download should work (we repeat this 20 times due to multiple work processes with apparently disconnected caches). Note that we use an API key instead of the password. I did not explicitly try this with a password.
 # Try the same again, but use an incorrect password (again 20 times)
 # Now retry with the correct password and obs

In [42]:
import pickle

In [43]:
data = {'model': model , 'ss_model': ss_model}
with open('saved_models.pkl', 'wb') as file:
    pickle.dump(data, file)

In [44]:
def test():
    return "Hello World"